In [1]:
from modules.scouter_handler import ScouterHandler

In [2]:
sh = ScouterHandler()

In [3]:
query_body = sh.make_keyword_query_body("문재인 대통령")

Query : 문재인 대통령


In [4]:
sh.search(query_body, data_type="newspaper")

Scroll idx : 1 (0 docs)
Trim lower 70% of docs (0 docs are cut)
Total retrieved Doc # :  0



[]

In [1]:
from modules.doc_graph_analyzer import DocGraphAnalyzer

In [2]:
a = DocGraphAnalyzer("가위")